In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col

spark = SparkSession.builder.appName("practical_test").getOrCreate()

23/03/19 01:33:22 WARN Utils: Your hostname, d5han-HP-ProBook-440-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.8.103 instead (on interface wlp2s0)
23/03/19 01:33:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/19 01:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [25]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import min, max

In [5]:
df_csv = pd.read_excel("../data/source/Online Retail.xlsx", sheet_name="Online Retail")
df_csv.to_csv("../data/csv_converted/Online Retail.csv", index=False)

In [7]:
df = spark.read.format("csv").option("header", "true").load("../data/csv_converted/Online Retail.csv")

In [8]:
df.head()


Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity='6', InvoiceDate='2010-12-01 08:26:00', UnitPrice='2.55', CustomerID='17850.0', Country='United Kingdom')

In [9]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [10]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [11]:
print((df.count(), len(df.columns)))

(541909, 8)


In [27]:
min_date, max_date = df.select(min("InvoiceDate"), max("InvoiceDate")).first()
min_date, max_date

('2010-12-01 08:26:00', '2011-12-09 12:50:00')

In [12]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [17]:
print(df.show(df.count(), False))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
countries_df = df.groupBy("Country").count().orderBy(col("count").desc())

In [37]:
countries_df.show(150, truncate = False)

+--------------------+------+
|Country             |count |
+--------------------+------+
|United Kingdom      |495478|
|Germany             |9495  |
|France              |8557  |
|EIRE                |8196  |
|Spain               |2533  |
|Netherlands         |2371  |
|Belgium             |2069  |
|Switzerland         |2002  |
|Portugal            |1519  |
|Australia           |1259  |
|Norway              |1086  |
|Italy               |803   |
|Channel Islands     |758   |
|Finland             |695   |
|Cyprus              |622   |
|Sweden              |462   |
|Unspecified         |446   |
|Austria             |401   |
|Denmark             |389   |
|Japan               |358   |
|Poland              |341   |
|Israel              |297   |
|USA                 |291   |
|Hong Kong           |288   |
|Singapore           |229   |
|Iceland             |182   |
|Canada              |151   |
|Greece              |146   |
|Malta               |127   |
|United Arab Emirates|68    |
|European 

In [38]:
countries_df.count()

38